# Spectra 1: Load, Plot, and Smooth an Optical Spectrum from SDSS

## Authors
Lia Corrales, Camilla Pacifici, Kelle Cruz

## Learning Goals
* Download an SDSS spectrum using `astroquery`
* Load the spectrum with `specutils`
* Write a function to display the spectrum in `matplotlib`
* Smooth the spectrum with `specutils`

## Keywords

spectroscopy, specutils, astroquery

## Companion Content

This tutorial is Part 1 of a series:
* **Spectra 1: Load, Plot, and Smooth an Optical Spectrum from SDSS**
* Spectra 2: Measure Properties of a Spectral Emission Line

These tutorials are based on the [MOS spectrum example](https://spacetelescope.github.io/jdat_notebooks/pages/mos-spectroscopy/mos-spectroscopy.html) by Camilla Pacifici.

More information about optical SDSS spectra can be found on the [Getting Started with the Optical Data](https://www.sdss.org/dr16/spectro/spectro_basics/) SDSS help page.


## Summary

In this tutorial, we will download the spectrum of a galaxy from the Sloan Digital Sky Survey (SDSS), load the spectrum as a Spectrum1D object using `specutils`, and then write a function to display the spectrum using `matplotlib`. We will use `specutils` to smooth the spectrum and plot it with the original.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u

from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord

from specutils import Spectrum1D
import specutils.manipulation

## Part 1: Download an SDSS spectrum

Here, we will follow the `astroquery.sdss` [documentation](https://astroquery.readthedocs.io/en/latest/sdss/sdss.html) in identifying an example SDSS spectrum for download.

First, we will query the SDSS archive for a quasar (QSO) spectrum at the following coordinates.


In [ ]:
object_coords = SkyCoord("0h8m05.63s +14d50m23.3s", frame='icrs')

xid = SDSS.query_region(object_coords, spectro=True)
xid

The `SDSS.query_region` function returns an Astropy Table object that stores database identification information. This object, which we've named `xid`, can now be passed to `SDSS.get_spectra` to download the spectrum from the archive.

In [ ]:
sp_hdus = SDSS.get_spectra(matches=xid)

The `SDSS.get_spectra` method returns a list of [HDUList](https://docs.astropy.org/en/stable/io/fits/api/hdulists.html) objects, so that each item of the list contains the contents of  FITS file. Because our query only returns one object, there is one HDUList returned by `SDSS.get_spectra`.

We can treat the contents of `sp_hdus` as if we loaded a FITS file. Check out the [Viewing and manipulating data from FITS tables tutorial](https://learn.astropy.org/rst-tutorials/FITS-tables.html?highlight=fits) to learn more about working with FITS files. For example, we can print the information about the content contained in the HDUList:

In [ ]:
print("Number of HDUList objects:", len(sp_hdus))

print(sp_hdus[0].info())

## Part 2: Load the SDSS spectrum into a `Spectrum1D` object

We will load the spectral information into a `specutils.Spectrum1D` object, which is the simplest case for using specutils.

### Method 1: Using the `specutils` built-in loader for SDSS spectrum files

Specutils has [many loaders](https://specutils.readthedocs.io/en/stable/spectrum1d.html#list-of-loaders) for common spectra formats, including one for SDSS spectra. If you have downloaded the SDSS spectrum FITS files, you can provide the filename to `Spectrum1D.read` and set the keyword argument `format="SDSS-III/IV spec"`.

In [ ]:
qso0008_spectrum = Spectrum1D.read(sp_hdus[0], format="SDSS-III/IV spec")

### Method 2: Creating a `Spectrum1D` object manually

Based on the [SDSS data model for spectra](https://data.sdss.org/datamodel/files/BOSS_SPECTRO_REDUX/RUN2D/spectra/PLATE4/spec.html), we suspect that the "COADD" extension holds the spectral information.

To make sure this is true, we will extract the table contents to a new variable and examine the column names.



In [ ]:
qso0008_coadd_data = sp_hdus[0]["COADD"].data

print(qso0008_coadd_data.columns)

Let's plot the columns that look like they might contain the spectrum information we want (wavelength and flux). We assume the 'flux' column contains the flux. Since the column name 'loglam' indicates that it contains the logarithim of lambda, a common name choice for wavelength, we'll plot that column after solving for lambda:

$$ \lambda = 10^{\log \lambda} $$

In [ ]:
lambda = np.power(10.0, qso0008_coadd_data['loglam'])
plt.plot(lambda, qso0008_coadd_data['flux'])

Excellent, this looks like a QSO spectrum! 

Based on the [Getting Started with the Optical Data](https://www.sdss.org/dr16/spectro/spectro_basics/) SDSS help page, the SDSS spectrograph covers the wavelength range 3800 - 9200 Angstrom. So the wavelength units for the 'loglam' column must be Angstrom. The website also states that the flux units are $10^{-17}$erg cm$^{-2}$ s$^{-1}$ Angstrom$^{-1}$.

In [ ]:
qso0008_spectrum = Spectrum1D(flux=qso0008_coadd_data['flux'] * 1.e-17 * u.Unit('erg cm^-2 s^-1 AA^-1'), 
                      spectral_axis=np.power(10.0, qso0008_coadd_data['loglam']) * u.angstrom)

## Part 3: Write a function to plot the spectrum

While it's possible to plot the spectrum directly by passing information stored in the `Spectrum1D` object to `matplotlib.pyplot.plot`, we will write a function for this process so that we can re-plot any `Spectrum1D` with ease.

In [ ]:
def plot_spectrum1d(ax, spec, **kwargs):
  """
  Plot a Spectrum1D object

  ax : matplotlib.pyplot.axis object on which to plot the data

  spec : specutils.Spectrum1D object to plot

  **kwargs will be passed to the matplotlib.pyplot.plot command
  """
  ax.plot(spec.spectral_axis, spec.flux, **kwargs)
  ax.set_xlabel(spec.spectral_axis.unit)
  ax.set_ylabel(spec.flux.unit)
  return

Next, set up the matplotlib axis and pass it to our newly defined function. We will then change the x-axis range in order to zoom in on some of the sharp emission features.

In [ ]:
ax = plt.subplot(111)

plot_spectrum1d(ax, qso0008_spectrum)
plt.xlim(5100, 5300)

Adding plotting functionality to specutils is currently under discussion and you can [read the details and contribute to the conversation on Github](https://github.com/astropy/specutils/pull/650).

## Part 4: Smooth the spectrum with `specutils` and re-plot

We can use a Gaussian kernel to smooth the spectrum data with the `specutils.manipulation.gaussian_smooth` function.

The standard deviation of the Gaussian function is defined in units of pixels (i.e., number of bins), with the `stddev` keyword.

In [ ]:
qso0008_smoothed_spectrum = specutils.manipulation.gaussian_smooth(qso0008_spectrum, stddev=5)

Now, using the function we defined above, let's plot our two spectra together so that we can see the difference between the original and smoothed spectrum.

In [ ]:
ax = plt.subplot(111)

plot_spectrum1d(ax, qso0008_spectrum, color='k', label='Original')
plot_spectrum1d(ax, qso0008_smoothed_spectrum, color='r', alpha=0.8, label='Smoothed')

plt.legend(loc='upper right')
plt.xlim(5100, 5300)

## Exercise

Modify the plotting function so you can plot the spectrum with a different x-axis unit, such as frequency. Your function definition might look like:

`def plot_spectrum1d(ax, spec, xunit='Angstrom'):`

**Hint:** Check out the [Astropy units equivalencies module](https://docs.astropy.org/en/stable/units/equivalencies.html#spectral-units) to learn how to manage conversions for spectral units.